In [43]:
!pip install -U tensorflow==2.11 

  Using cached tensorflow-2.11.0-cp39-cp39-win_amd64.whl (1.9 kB)
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.12.0
    Uninstalling tensorflow-2.12.0:
      Successfully uninstalled tensorflow-2.12.0


In [44]:
!pip show tensorflow

Name: tensorflow
Version: 2.11.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\public\new\envs\tensorflowraul\lib\site-packages
Requires: tensorflow-intel
Required-by: 


In [52]:
# TensorFlow and TF-Hub modules.
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed

logging.set_verbosity(logging.ERROR)

# Some modules to help with reading the UCF101 dataset.
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np

# Some modules to display an animation using imageio.
import imageio
from IPython import display

from urllib import request  # requires python3

In [53]:
###
###
###

In [54]:
def frames_from_video_file(video_path, n_frames, output_size = (224,224), frame_step = 15):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  # Read each video frame by frame
  result = []
  src = cv2.VideoCapture(str(video_path))  

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

In [55]:
def format_frames(frame, output_size):
  """
    Pad and resize an image from a video.

    Args:
      frame: Image that needs to resized and padded. 
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

In [56]:
###
###
###

In [57]:
def list_videos(root_path):
    """
    Returns a list of videos and their categories.
    """
    videos = []
    for set_type in os.listdir(root_path):
        set_path = os.path.join(root_path, set_type)
        if not os.path.isdir(set_path):
            continue
        for category in os.listdir(set_path):
            category_path = os.path.join(set_path, category)
            if not os.path.isdir(category_path):
                continue
            for video in os.listdir(category_path):
                video_path = os.path.join(category_path, video)
                if not video_path.endswith(".mp4"):
                    continue
                videos.append((video_path, set_type, category))
    return videos

videos = list_videos(r"C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips")

categories = {}
for video in videos:
    category = video[2]
    if category not in categories:
        categories[category] = []
    categories[category].append(video[0])
print("Found %d videos in %d categories." % (len(videos), len(categories)))

for category, sequences in categories.items():
    summary = ", ".join(sequences[:2])
    print("%-20s %4d videos (%s, ...)" % (category, len(sequences), summary))

Found 200 videos in 6 categories.
background             71 videos (C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\background\background_13.mp4, C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\background\background_31.mp4, ...)
IntCleaner             15 videos (C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\IntCleaner\IntCleaner_4.mp4, C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\IntCleaner\IntCleaner_6.mp4, ...)
IntClient              75 videos (C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\IntClient\IntClient_15.mp4, C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\IntClient\IntClient_17_1.mp4, ...)
Jolt                   19 videos (C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\Jolt\Jolt_12.mp4, C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\Jolt\Jolt_4.mp4, ...)
TS                     18 videos (C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\TS\TS_6

In [58]:
video_path = r"C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\train\Run\Run_0_1.mp4"
sample_video = frames_from_video_file(video_path, n_frames = 10)
sample_video.shape

(10, 224, 224, 3)

In [59]:
i3d = hub.load("https://tfhub.dev/deepmind/i3d-kinetics-400/1").signatures['default']

In [60]:
labels = list(categories.keys())

In [61]:
def predict(sample_video):
  # Add a batch axis to the sample video.
  model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

  logits = i3d(model_input)['default'][0]
  probabilities = tf.nn.softmax(logits)

  print("Top 5 actions:")
  for i in np.argsort(probabilities)[::-1][:5]:
    print(f"  {labels[i]:22}: {probabilities[i] * 100:5.2f}%")

In [62]:
predict(sample_video)

Top 5 actions:


IndexError: list index out of range

In [63]:
### está a utilizar os labels do modelo original